# Trabajo de Inteligencia Artificial

## Análisis de sentimientos

Realizado por:
- Alicia Sánchez Hossdorf
- Rafael Segura Gómez

Fecha: 26/05/2023

Convocatoria de Junio

<h1> 1. Recopilación de datos. </h1>

In [59]:
import pandas
import numpy
from textblob import TextBlob

Hemos descargado todos los tweets que necesitamos en un CSV. Por lo que debemos leer los datos del archivo.

In [60]:
#tweetsCSV = pandas.read_csv("../Notebook/Tweets/Tweets_entrenamiento.csv", header=0)
tweetsCSV = pandas.read_csv("../Notebook/Tweets/Tweets_entrenamiento_small.csv", header=0)

Para saber cuantos tweets tenemos, miraremos cuantas filas tiene el archivo.

In [61]:
print(tweetsCSV.shape)

(78, 4)


In [62]:
# 10 primeras filas
tweetsCSV.head(10)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
7,50e14c0bb8,Soooo high,Soooo high,neutral
8,e050245fbd,Both of you,Both of you,neutral
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive


Como solo necesitamos la columna text, que es donde viene los tweets que necesitamos, haremos una seleccion de dicha columna.


In [63]:
tweets = tweetsCSV[['text']]

In [64]:
tweets.head(20)

,text
0,"I`d have responded, if I were going"
1,Sooo SAD I will miss you here in San Diego!!!
2,my boss is bullying me...
3,what interview! leave me alone
4,"Sons of ****, why couldn`t they put them on t..."
5,http://www.dothebouncy.com/smf - some shameles...
6,2am feedings for the baby are fun when he is a...
7,Soooo high
8,Both of you
9,Journey!? Wow... u just became cooler. hehe....


<h1> 2. Eliminar las palabras que no aportan información </h1>

In [65]:
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer

In [81]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [67]:
from nltk import download

download('punkt', download_dir='.')

[nltk_data] Downloading package punkt to ....
[nltk_data]   Package punkt is already up-to-date!


True

In [68]:
sWords = set(stopwords.words('english'))
print(sWords)

{'having', 'any', 'been', 'through', 'we', 'or', 'there', 'ourselves', 'hasn', 'as', 'nor', 'each', 'are', 'again', "she's", 'wouldn', 'only', "wouldn't", 'did', 'to', 'isn', 'be', 'by', 'into', "wasn't", 'those', 'this', 'i', 'ours', 'my', 've', "isn't", "it's", 'on', 'out', 'mightn', "mightn't", "weren't", "hasn't", 'below', 'myself', 'very', 'because', "won't", 'do', 'so', 'after', 'during', 'of', 'here', 'should', 'himself', 'your', 'off', 'against', 'between', 'up', 'same', 'most', 'theirs', 'her', 'were', 'then', 'doing', "needn't", "you're", 'its', 'has', 'ma', 'in', 'll', 'she', 'above', "doesn't", 'but', 'am', 'weren', 'further', 'few', 'just', 'if', 'that', 'you', 'which', 'where', "don't", 'wasn', 'and', 'our', 'who', "shan't", 'whom', 'about', "aren't", 'the', "hadn't", 'ain', 'what', 'all', 'needn', 'at', 'o', 'until', 'haven', 'once', 'both', 'such', 'm', 'yourselves', "that'll", 'yourself', 'while', 'own', 'themselves', 'aren', "didn't", 'down', 'over', 'too', 'a', 't', 

Crear una función clean_text para eliminar las palabras comunes y poco informativas; eliminar menciones, hashtags, URLs y cualquier otro símbolo extraño; y que utilice técnicas de lematización (stemming) y tokenización para reducir las palabras a su forma base

In [69]:
#Funcion para eliminar las stop words y tokenizado.

def delete_stopWords(tweet):
    tweetLow = tweet.lower()
    tweetWords = word_tokenize(tweetLow)
    tweetFiltered = []
        
    for w in tweetWords:
        if w not in sWords:
            tweetFiltered.append(w)
    return tweetFiltered

In [70]:
#Esta funcion es para eliminar menciones, hashtags, URLs y cualquier otro símbolo extraño.

def delete_otherElements(tweet):
    tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
    tweet = re.sub("#[A-Za-z0-9_]+","", tweet)
    tweet = re.sub("http[^\s]+","",tweet)
    tweetFiltered = re.sub(r"[-()\"#/@;:<>{}+=*~|.!?,]", "", tweet)

    return tweetFiltered
    

## Función para limpiar el text

Con los tweets tokenizados, lematizados y sin stop words lo pasamos a lista.

In [71]:
def clean_text(Tweets): 
    res = Tweets.copy()
    i = 0;
    #Iniciar SnowBall Stemmer con idioma ingles
    stemmer = SnowballStemmer("english")
    while i<len(tweets):
        tweet_cleaned = []
        
        clean_t = delete_otherElements(Tweets['text'].get(i))
        
        #__Para corregir gramaticalmente descomentar la siguiente linea__
        #clean_t = str(TextBlob(clean_t).correct())
        
        tNoStopWords = delete_stopWords(clean_t)
        #Esto es para la lematización de la frase. 
        for w in tNoStopWords:
            if w != "`":
                tweet_cleaned.append(stemmer.stem(w))
        res.at[i,'text'] = tweet_cleaned
        
        i+=1
    return res

In [72]:
tweets_cleaned = clean_text(tweets)

In [73]:
print(tweets_cleaned.head(20))

                                                 text
0                                       [respond, go]
1                       [sooo, sad, miss, san, diego]
2                                       [boss, bulli]
3                             [interview, leav, alon]
4                 [son, put, releas, alreadi, bought]
5       [shameless, plug, best, ranger, forum, earth]
6                  [2am, feed, babi, fun, smile, coo]
7                                       [soooo, high]
8                                                  []
9     [journey, wow, u, becam, cooler, hehe, possibl]
10  [much, love, hope, reckon, chanc, minim, p, ne...
11  [realli, realli, like, song, love, stori, tayl...
12                    [sharpi, run, danger, low, ink]
13             [want, go, music, tonight, lost, voic]
14                             [test, test, lg, env2]
15                                  [uh, oh, sunburn]
16               [ok, tri, plot, altern, speak, sigh]
17  [sick, past, day, thus, 

### A partir de aquí, hasta el etiquetado de datos, solamente es prueba de corrección gramatical, se puede ignorar, son pruebas

In [74]:
# Corrige el dataframe y muestra ambas columnas, corregido y original, no modifica variables originales
tweets_cleaned['text_corregido'] = tweets_cleaned['text'].apply(lambda x: str(TextBlob(" ".join(x)).correct()))


print(tweets_cleaned.head(20))

                                                 text  \
0                                       [respond, go]   
1                       [sooo, sad, miss, san, diego]   
2                                       [boss, bulli]   
3                             [interview, leav, alon]   
4                 [son, put, releas, alreadi, bought]   
5       [shameless, plug, best, ranger, forum, earth]   
6                  [2am, feed, babi, fun, smile, coo]   
7                                       [soooo, high]   
8                                                  []   
9     [journey, wow, u, becam, cooler, hehe, possibl]   
10  [much, love, hope, reckon, chanc, minim, p, ne...   
11  [realli, realli, like, song, love, stori, tayl...   
12                    [sharpi, run, danger, low, ink]   
13             [want, go, music, tonight, lost, voic]   
14                             [test, test, lg, env2]   
15                                  [uh, oh, sunburn]   
16               [ok, tri, plot

In [75]:
# Muestra el dataframe con los textos corregidos
tweets_cleaned[['text_corregido']]

,text_corregido
0,respond go
1,soon sad miss san diego
2,boss bull
3,interview leave along
4,son put release already bought
...,...
73,you ride one catch one summer til pop open one '
74,good gorjuz yea no ask yesterday the hospital ...
75,ok pop say hi check thing probably head gutta ...
76,now really mission family today added


## Etiquetado de datos

Documentar:
- !pip install textblob

In [76]:
import os
import csv
from textblob import TextBlob

In [113]:

def tag_text(text):
    tweet = ' '.join(text)  # Convertir la lista de palabras en una cadena de text
    blob = TextBlob(tweet) #Convierte los tweets a tipo Blob para aplicarle la libreria
    tag = blob.sentiment.polarity #Polaridad entre -1 y 1, de peor a mejor respectivamente
    return tweet, tag

csv_path = 'Tweets/resultados.csv'

#Escritura en un archivo CSV nuevo
with open(csv_path, 'w', newline='',encoding='utf-8-sig') as csv_file: #Forzamos formato utf-8, otro da problemas
    writer = csv.writer(csv_file)
    writer.writerow(['text', 'tag', 'sentiment']) #Crea columna sentiment también
    
    if len(tweets_cleaned) == 0:  #Checkea si está vacío
        print("Nothing to tag, espabila notas.")
        exit()

    for _, row in tweets_cleaned.iterrows(): #Usamos iterrows porque tweets_cleaned es de tipo dataframe
        result = tag_text(row['text'])
        result = (*result, '')  # Añadir una cadena vacía para la columna "sentiment"
        writer.writerow(result)


In [122]:
#Categorizando los tags a valores discretos
def etiqueta_codificada(etiqueta):
    if etiqueta >= -1 and etiqueta < -0.5:
        return "hater"
    elif etiqueta >= -0.5 and etiqueta < 0:
        return "molesto"
    elif etiqueta == 0:
        return "neutro"
    elif etiqueta > 0 and etiqueta <= 0.5:
        return "contento"
    elif etiqueta > 0.5 and etiqueta <= 1:
        return "muy feliz"
    else:
        return "null"


In [124]:
#Actualizar la columna sentiment del csv
# Leer el archivo CSV
csv_path = 'Tweets/resultados.csv'
df = pandas.read_csv(csv_path)

# Aplicar la función etiqueta_codificada para asignar los valores de tag a sentiment
df['sentiment'] = df['tag'].apply(etiqueta_codificada)

# Guardar el archivo CSV actualizado
df.to_csv(csv_path, index=False)

In [125]:
tweets_results = pandas.read_csv("../Notebook/Tweets/resultados.csv", header=0)

In [126]:
print(tweets_results.shape)

(78, 3)


In [127]:
# 10 primeras filas
tweets_results.head(20)

,text,tag,sentiment
0,respond go,0.000000,neutro
1,sooo sad miss san diego,-0.500000,molesto
2,boss bulli,0.000000,neutro
3,interview leav alon,0.000000,neutro
4,son put releas alreadi bought,0.000000,neutro
5,shameless plug best ranger forum earth,1.000000,muy feliz
6,2am feed babi fun smile coo,0.300000,contento
7,soooo high,0.160000,contento
8,NaN,0.000000,neutro
9,journey wow u becam cooler hehe possibl,0.100000,contento


### Entrenamiento del modelo
Necesitamos separar los datos que tenemos en conjunto de entrenamiento y conjunto de pruebas, para probar que funciona bien e ir entrenando el modelo.

A partir de aqui estoy de pruebas, nada de ésto funciona, aunque algunas cosas hacen cosas

In [131]:
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder


In [129]:
atributos = tweets_results.loc[:, 'tag']  # selección de las columnas de atributos
objetivo = tweets_results['sentiment']  # selección de la columna objetivo

In [130]:
codificador_atributos = preprocessing.OrdinalEncoder() # --> transforma de discreta a ordinal
codificador_atributos.fit(atributos) # --> entrena el modelo
# Categorías detectadas por el codificador para cada atributo
print(codificador_atributos.categories_)

ValueError: Expected 2D array, got 1D array instead:
array=[ 0.         -0.5         0.          0.          0.          1.
  0.3         0.16        0.          0.1         0.5         0.5
  0.          0.          0.          0.          0.5        -0.48214286
  0.          0.          0.          0.06818182  0.18333333  0.
 -0.3125      0.35        0.          0.          0.3         0.
  0.13636364  0.          0.2         0.5         0.3         0.
 -0.05681818  0.13636364  0.          0.46666667  0.          0.375
  0.         -0.2         0.5         0.1        -0.29166667  0.
 -0.2         0.06818182 -0.8         0.          0.         -0.25
  0.          0.         -0.3         0.          0.          0.3
 -0.2         0.2         0.          0.275       0.          0.
  0.7         0.28571429  0.          0.          0.          0.56666667
  0.          0.          0.7         0.25        0.1         0.        ].
Reshape your data either using array.reshape(-1, 1) if your dat